<a href="https://colab.research.google.com/github/massegu/Percepcion_V1_Clinica/blob/main/ON_OFF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

# Generador de estímulos
def generar_estimulo(tipo):
    matriz = np.zeros((5,5))
    if tipo == "centro_brillante":
        matriz[2,2] = 1
    elif tipo == "centro_oscuro":
        matriz = np.ones((5,5))
        matriz[2,2] = 0
    elif tipo == "periferia_brillante":
        matriz = np.ones((5,5))
        matriz[2,2] = 0
    elif tipo == "periferia_oscura":
        matriz = np.zeros((5,5))
        matriz[2,2] = 1
    elif tipo == "uniforme_brillante":
        matriz = np.ones((5,5))
    elif tipo == "uniforme_oscuro":
        matriz = np.zeros((5,5))
    return matriz

# Modelos de respuesta
def respuesta_bipolar(matriz, tipo):
    centro = matriz[2,2]
    return centro if tipo == "ON" else 1 - centro

def respuesta_ganglionar(matriz, tipo):
    centro = matriz[2,2]
    periferia = np.mean(np.delete(matriz.flatten(), 12))
    return centro - periferia if tipo == "ON" else periferia - centro

# Campo receptivo esquemático
def generar_campo_receptivo(tipo_celula):
    receptivo = np.zeros((5,5))
    centro = (2,2)
    for i in range(5):
        for j in range(5):
            if (i,j) == centro:
                receptivo[i,j] = 1 if tipo_celula == "ON" else -1
            else:
                receptivo[i,j] = -1 if tipo_celula == "ON" else 1
    return receptivo

# Visualización completa
def visualizar_completo(tipo):
    matriz = generar_estimulo(tipo)
    respuestas = {
        "Bipolar ON": respuesta_bipolar(matriz, "ON"),
        "Bipolar OFF": respuesta_bipolar(matriz, "OFF"),
        "Ganglionar ON": respuesta_ganglionar(matriz, "ON"),
        "Ganglionar OFF": respuesta_ganglionar(matriz, "OFF")
    }

    fig, axs = plt.subplots(1, 3, figsize=(18,5))

    # Estímulo visual
    axs[0].imshow(matriz, cmap='gray', interpolation='nearest')
    axs[0].set_title(f"🖼 Estímulo aplicado: {tipo.replace('_',' ')}")
    axs[0].axis('off')

    # Respuesta celular
    axs[1].bar(respuestas.keys(), respuestas.values(), color=['skyblue','salmon','limegreen','orange'])
    axs[1].set_title("📊 Activación de células visuales")
    axs[1].set_ylabel("Nivel de activación")
    axs[1].set_ylim(-1,1)
    axs[1].grid(True)

    # Campo receptivo ganglionar ON
    receptivo = generar_campo_receptivo("ON")
    im = axs[2].imshow(receptivo, cmap='bwr', vmin=-1, vmax=1)
    axs[2].set_title("🧠 Campo receptivo ganglionar ON\nCentro (rojo) vs Periferia (azul)")
    axs[2].axis('off')
    fig.colorbar(im, ax=axs[2], orientation='vertical', label='Activación relativa')

    plt.tight_layout()
    plt.show()

# Widget interactivo
selector = widgets.Dropdown(
    options=["centro_brillante", "centro_oscuro", "periferia_brillante", "periferia_oscura", "uniforme_brillante", "uniforme_oscuro"],
    description='Estímulo:',
)

widgets.interact(visualizar_completo, tipo=selector)


interactive(children=(Dropdown(description='Estímulo:', options=('centro_brillante', 'centro_oscuro', 'perifer…

<function __main__.visualizar_completo(tipo)>

### 🧠 Tipos de células y su función

- **Célula bipolar ON**  
  Se activa cuando hay luz en el centro del campo receptivo. Detecta incrementos de luminancia.

- **Célula bipolar OFF**  
  Se activa cuando hay oscuridad en el centro. Detecta decrementos de luminancia.

- **Célula ganglionar ON (Centro ON - Periferia OFF)**  
  Se activa cuando el centro está iluminado y la periferia oscura. Detecta bordes brillantes.

- **Célula ganglionar OFF (Centro OFF - Periferia ON)**  
  Se activa cuando el centro está oscuro y la periferia iluminada. Detecta bordes oscuros.

🔍 Estas células trabajan en conjunto para detectar **contrastes locales**, **bordes**, y **transiciones de luz**, fundamentales para la percepción visual.


🧠 Versión 1: Campo receptivo dinámico según el estímulo

Esta versión ajusta el esquema del campo receptivo según cuál de las células ganglionares (ON u OFF) responde más al estímulo aplicado.

In [8]:
def visualizar_dinamico(tipo):
    matriz = generar_estimulo(tipo)
    respuestas = {
        "Bipolar ON": respuesta_bipolar(matriz, "ON"),
        "Bipolar OFF": respuesta_bipolar(matriz, "OFF"),
        "Ganglionar ON": respuesta_ganglionar(matriz, "ON"),
        "Ganglionar OFF": respuesta_ganglionar(matriz, "OFF")
    }

    tipo_dominante = "ON" if respuestas["Ganglionar ON"] >= respuestas["Ganglionar OFF"] else "OFF"

    fig, axs = plt.subplots(1, 3, figsize=(18,5))

    axs[0].imshow(matriz, cmap='gray')
    axs[0].set_title(f"Estímulo: {tipo.replace('_',' ')}")
    axs[0].axis('off')

    axs[1].bar(respuestas.keys(), respuestas.values(), color=['skyblue','salmon','limegreen','orange'])
    axs[1].set_title("Activación celular")
    axs[1].set_ylim(-1,1)
    axs[1].grid(True)

    receptivo = generar_campo_receptivo(tipo_dominante)
    im = axs[2].imshow(receptivo, cmap='bwr', vmin=-1, vmax=1)
    axs[2].set_title(f"Campo receptivo ganglionar {tipo_dominante}")
    axs[2].axis('off')
    fig.colorbar(im, ax=axs[2], orientation='vertical')

    plt.tight_layout()
    plt.show()

widgets.interact(visualizar_dinamico, tipo=selector)

interactive(children=(Dropdown(description='Estímulo:', options=('centro_brillante', 'centro_oscuro', 'perifer…

<function __main__.visualizar_dinamico(tipo)>

Versión 2: Campos receptivos ON y OFF en paralelo
Esta versión muestra ambos esquemas de campo receptivo simultáneamente, para facilitar la comparación estructural.

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import clear_output

# Generador de estímulos
def generar_estimulo(tipo):
    matriz = np.zeros((5,5))
    if tipo == "centro_brillante":
        matriz[2,2] = 1
    elif tipo == "centro_oscuro":
        matriz = np.ones((5,5))
        matriz[2,2] = 0
    elif tipo == "periferia_brillante":
        matriz = np.ones((5,5))
        matriz[2,2] = 0
    elif tipo == "periferia_oscura":
        matriz = np.zeros((5,5))
        matriz[2,2] = 1
    elif tipo == "uniforme_brillante":
        matriz = np.ones((5,5))
    elif tipo == "uniforme_oscuro":
        matriz = np.zeros((5,5))
    return matriz

# Modelos de respuesta
def respuesta_bipolar(matriz, tipo):
    centro = matriz[2,2]
    return centro if tipo == "ON" else 1 - centro

def respuesta_ganglionar(matriz, tipo):
    centro = matriz[2,2]
    periferia = np.mean(np.delete(matriz.flatten(), 12))
    return centro - periferia if tipo == "ON" else periferia - centro

# Campo receptivo esquemático
def generar_campo_receptivo(tipo_celula):
    receptivo = np.zeros((5,5))
    centro = (2,2)
    for i in range(5):
        for j in range(5):
            if (i,j) == centro:
                receptivo[i,j] = 1 if tipo_celula == "ON" else -1
            else:
                receptivo[i,j] = -1 if tipo_celula == "ON" else 1
    return receptivo

# Visualización completa
def visualizar_comparativo(tipo):
    clear_output(wait=True)
    matriz = generar_estimulo(tipo)
    respuestas = {
        "Bipolar ON": respuesta_bipolar(matriz, "ON"),
        "Bipolar OFF": respuesta_bipolar(matriz, "OFF"),
        "Ganglionar ON": respuesta_ganglionar(matriz, "ON"),
        "Ganglionar OFF": respuesta_ganglionar(matriz, "OFF")
    }

    fig, axs = plt.subplots(1, 4, figsize=(22,5))

    # Estímulo visual
    axs[0].imshow(matriz, cmap='gray')
    axs[0].set_title(f"Estímulo aplicado: {tipo.replace('_',' ')}")
    axs[0].axis('off')

    # Respuesta celular
    axs[1].bar(respuestas.keys(), respuestas.values(), color=['skyblue','salmon','limegreen','orange'])
    axs[1].set_title("Activación celular")
    axs[1].set_ylim(-1,1)
    axs[1].grid(True)

    # Campo receptivo ON
    receptivo_on = generar_campo_receptivo("ON")
    im1 = axs[2].imshow(receptivo_on, cmap='bwr', vmin=-1, vmax=1)
    axs[2].set_title("Campo receptivo ganglionar ON")
    axs[2].axis('off')
    fig.colorbar(im1, ax=axs[2], orientation='vertical')

    # Campo receptivo OFF
    receptivo_off = generar_campo_receptivo("OFF")
    im2 = axs[3].imshow(receptivo_off, cmap='bwr', vmin=-1, vmax=1)
    axs[3].set_title("Campo receptivo ganglionar OFF")
    axs[3].axis('off')
    fig.colorbar(im2, ax=axs[3], orientation='vertical')

    plt.tight_layout()
    plt.show()

# Widget interactivo
selector = widgets.Dropdown(
    options=["centro_brillante", "centro_oscuro", "periferia_brillante", "periferia_oscura", "uniforme_brillante", "uniforme_oscuro"],
    description='Estímulo:',
)

widgets.interact(visualizar_comparativo, tipo=selector)




interactive(children=(Dropdown(description='Estímulo:', options=('centro_brillante', 'centro_oscuro', 'perifer…

<function __main__.visualizar_comparativo(tipo)>